In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [1]:
import tensorflow as tf

In [2]:
x = tf.random.normal(shape = (1, 10, 1))

In [5]:
tf.image.resize(x, [1, 16])

<tf.Tensor 'resize_2/Squeeze:0' shape=(1, 20, 1) dtype=float32>

In [6]:
sess = tf.Session()

In [12]:
sess.run([tf.image.resize(x, [1, 10], method = 'nearest'), x])

[array([[[-0.50713724],
         [-0.43177643],
         [ 0.03711988],
         [ 1.6818367 ],
         [-0.66351444],
         [-0.6343261 ],
         [ 0.8212245 ],
         [-0.63036644],
         [-1.2326638 ],
         [-0.075799  ]]], dtype=float32),
 array([[[-0.50713724],
         [-0.43177643],
         [ 0.03711988],
         [ 1.6818367 ],
         [-0.66351444],
         [-0.6343261 ],
         [ 0.8212245 ],
         [-0.63036644],
         [-1.2326638 ],
         [-0.075799  ]]], dtype=float32)]

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import pickle

metadata = pickle.load(open('metadata.pkl', "rb"))

In [4]:
import numpy as np
from math import ceil
import tensorflow as tf
import malaya_speech
from malaya_speech.train.model import fastspeech, autovc_fast






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





In [5]:
def pad_seq(x, base = 32):
    len_out = int(base * ceil(float(x.shape[0])/base))
    len_pad = len_out - x.shape[0]
    assert len_pad >= 0
    return np.pad(x, ((0,len_pad),(0,0)), 'constant'), len_pad

In [6]:
sbmt_i = metadata[0]
x_org = sbmt_i[2]
x_org, len_pad = pad_seq(x_org)
c = np.random.normal(size = (2, 512)).astype(np.float32)

In [7]:
x_org.shape

(96, 80)

In [8]:
len_pad

6

In [9]:
mel = tf.placeholder(tf.float32, (None, None, 80))
ori_vector = tf.placeholder(tf.float32, (None, 512))
target_vector = tf.placeholder(tf.float32, (None, 512))
mel_lengths = tf.placeholder(tf.int32, (None))

In [10]:
mel_lengths

<tf.Tensor 'Placeholder_3:0' shape=<unknown> dtype=int32>

In [11]:
dim_neck = 32
dim_speaker = 512
config = malaya_speech.config.fastspeech_config
config['encoder_hidden_size'] = dim_speaker + 80
config['decoder_hidden_size'] = dim_speaker + dim_neck
config = fastspeech.Config(vocab_size = 1, **config)

In [12]:
model = autovc_fast.Model(16, 512, 16, config)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [13]:
encoder_outputs, mel_before, mel_after, codes = model(mel, ori_vector, target_vector, mel_lengths)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [14]:
codes_ = model.call_second(mel_after, ori_vector)

In [15]:
encoder_outputs, mel_before, mel_after, codes, codes_

(<tf.Tensor 'autovc/concat:0' shape=(?, ?, 544) dtype=float32>,
 <tf.Tensor 'autovc/add_3:0' shape=(?, ?, 80) dtype=float32>,
 <tf.Tensor 'autovc/add_5:0' shape=(?, ?, 80) dtype=float32>,
 <tf.Tensor 'autovc/Encoder/TensorArrayStack/TensorArrayGatherV3:0' shape=(?, ?, 32) dtype=float32>,
 <tf.Tensor 'Encoder/TensorArrayStack/TensorArrayGatherV3:0' shape=(?, ?, 32) dtype=float32>)

In [16]:
loss_f = tf.losses.mean_squared_error
max_length = tf.cast(tf.reduce_max(mel_lengths), tf.int32)
mask = tf.sequence_mask(
    lengths = mel_lengths, maxlen = max_length, dtype = tf.float32
)
mask = tf.expand_dims(mask, axis = -1)
mask

<tf.Tensor 'ExpandDims:0' shape=<unknown> dtype=float32>

In [17]:
tf.sequence_mask(
    lengths = mel_lengths, maxlen = max_length, dtype = tf.float32
)

<tf.Tensor 'SequenceMask_1/Cast_1:0' shape=<unknown> dtype=float32>

In [18]:
from functools import partial
from malaya_speech.train.loss import calculate_2d_loss, calculate_3d_loss

mse_mel = partial(
    loss_f,
    weights = mask
)

In [19]:
mel_loss_before = calculate_3d_loss(mel, mel_before, mse_mel)
mel_loss_before

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


<tf.Tensor 'mean_squared_error/value:0' shape=() dtype=float32>

In [20]:
mel_loss_after = calculate_3d_loss(mel, mel_after, mse_mel)
mel_loss_after

<tf.Tensor 'mean_squared_error_1/value:0' shape=() dtype=float32>

In [21]:
g_loss_cd = tf.losses.absolute_difference(codes, codes_)

In [22]:
loss = mel_loss_before + mel_loss_after + g_loss_cd

In [23]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [34]:
o = sess.run([encoder_outputs, mel_before, mel_after], feed_dict = {mel: [x_org] * 2,
                                                                    ori_vector: c,
                                                                    target_vector: c,
                                                                    mel_lengths: [len(x_org)] * 2})

In [32]:
x_org[0]

array([0.39463845, 0.28340304, 0.26415357, 0.2532913 , 0.23593669,
       0.27531558, 0.2601337 , 0.20799784, 0.17925   , 0.16225114,
       0.21628033, 0.19391258, 0.16690125, 0.23933716, 0.26766717,
       0.23639701, 0.16513656, 0.24623321, 0.2517523 , 0.25785658,
       0.14969628, 0.24087207, 0.33559245, 0.18417373, 0.28367934,
       0.31905246, 0.27308086, 0.27488646, 0.33587086, 0.3004063 ,
       0.2561785 , 0.28301883, 0.282115  , 0.19620728, 0.15127781,
       0.20626858, 0.22624898, 0.24989904, 0.24906628, 0.20903952,
       0.19618066, 0.22475033, 0.21816216, 0.2561646 , 0.22669327,
       0.29271293, 0.274277  , 0.23617537, 0.20653298, 0.2006858 ,
       0.29317287, 0.31299165, 0.25377366, 0.26970923, 0.2563963 ,
       0.29354194, 0.27708203, 0.25875154, 0.25198594, 0.24439889,
       0.2782202 , 0.26452845, 0.2619485 , 0.27706414, 0.2742338 ,
       0.25750852, 0.19870667, 0.21669863, 0.17528968, 0.19980377,
       0.24140976, 0.24160649, 0.2322267 , 0.2224023 , 0.20069

In [36]:
o[1]

array([[[ 3.94638479e-01,  2.83403009e-01,  2.64153600e-01, ...,
          2.38987952e-01,  2.21619144e-01,  1.74386799e-01],
        [ 3.96624953e-01,  3.27039301e-01,  2.58440435e-01, ...,
          2.40468159e-01,  2.21521184e-01,  1.80433363e-01],
        [ 4.12980884e-01,  3.14762771e-01,  2.50771791e-01, ...,
          2.22262159e-01,  2.07013756e-01,  1.88533172e-01],
        ...,
        [ 2.80758954e-08, -1.02210640e-08,  6.75946898e-09, ...,
          1.94705692e-08,  1.10775993e-08, -9.87868631e-09],
        [ 2.13840785e-08, -1.53924751e-09,  9.29438393e-09, ...,
          2.35977549e-08,  5.63882940e-09, -1.52365676e-08],
        [ 3.29520375e-08, -1.33449589e-08,  3.56345109e-09, ...,
          1.22765496e-08,  3.60267904e-09, -1.52606852e-08]],

       [[ 3.94638419e-01,  2.83403039e-01,  2.64153570e-01, ...,
          2.38987952e-01,  2.21619129e-01,  1.74386799e-01],
        [ 3.96624893e-01,  3.27039331e-01,  2.58440435e-01, ...,
          2.40468159e-01,  2.21521169e

In [27]:
o[0].shape, o[1].shape, o[2].shape

((2, 96, 544), (2, 96, 80), (2, 96, 80))

In [28]:
o = sess.run([mel_loss_before, mel_loss_after, g_loss_cd], feed_dict = {mel: [x_org] * 2,
                                                                    ori_vector: c,
                                                                    target_vector: c,
                                                                    mel_lengths: [len(x_org)] * 2})

In [29]:
o

[1.6455271e-16, 3.1047273e-16, 0.06792629]

In [30]:
tf.trainable_variables()

[<tf.Variable 'autovc/Encoder/sequential/ConvNorm/conv1d/kernel:0' shape=(5, 592, 512) dtype=float32>,
 <tf.Variable 'autovc/Encoder/sequential/ConvNorm/conv1d/bias:0' shape=(512,) dtype=float32>,
 <tf.Variable 'autovc/Encoder/sequential/batch_normalization/gamma:0' shape=(512,) dtype=float32>,
 <tf.Variable 'autovc/Encoder/sequential/batch_normalization/beta:0' shape=(512,) dtype=float32>,
 <tf.Variable 'autovc/Encoder/sequential_1/ConvNorm/conv1d_1/kernel:0' shape=(5, 512, 512) dtype=float32>,
 <tf.Variable 'autovc/Encoder/sequential_1/ConvNorm/conv1d_1/bias:0' shape=(512,) dtype=float32>,
 <tf.Variable 'autovc/Encoder/sequential_1/batch_normalization_1/gamma:0' shape=(512,) dtype=float32>,
 <tf.Variable 'autovc/Encoder/sequential_1/batch_normalization_1/beta:0' shape=(512,) dtype=float32>,
 <tf.Variable 'autovc/Encoder/sequential_2/ConvNorm/conv1d_2/kernel:0' shape=(5, 512, 512) dtype=float32>,
 <tf.Variable 'autovc/Encoder/sequential_2/ConvNorm/conv1d_2/bias:0' shape=(512,) dtype=f

In [27]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, "test/model.ckpt")

'test/model.ckpt'

In [28]:
!ls -lh test

total 218496
-rw-r--r--  1 huseinzolkepli  staff    77B Feb 12 01:23 checkpoint
-rw-r--r--  1 huseinzolkepli  staff    97M Feb 12 01:23 model.ckpt.data-00000-of-00001
-rw-r--r--  1 huseinzolkepli  staff   4.6K Feb 12 01:23 model.ckpt.index
-rw-r--r--  1 huseinzolkepli  staff    10M Feb 12 01:23 model.ckpt.meta


In [29]:
!rm -rf test